# Installation

In [1]:
%%bash
pip install  --upgrade\
    'vllm>=0.8.2' \
    'transformers>=4.50.3' \
    pyzmq \
    unsloth \
    accelerate \
    bitsandbytes \
    openai \
    langchain-text-splitters \
    peft \
    FlagEmbedding \
    datasets \
    faiss-cpu \
    "flashinfer-python>=0.2.4"  --extra-index-url https://flashinfer.ai/whl/cu124/torch2.6/
git clone https://github.com/ggml-org/llama.cpp.git
cd llama.cpp/gguf-py/ && pip install --editable .
pip install jupyter-kernel-gateway ipykernel
pip install --upgrade --no-deps numpy==1.26.4 pandas==2.2.2

Looking in indexes: https://pypi.org/simple, https://flashinfer.ai/whl/cu124/torch2.6/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.2/544.2 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
Cloning into 'llama.cpp'...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.8.2 requires gguf==0.10.0, but you have gguf 0.16.0 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 6.5.7 requi

In [2]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_WRITE_TOKEN')
!huggingface-cli login --add-to-git-credential --token $HF_TOKEN

Token is valid (permission: write).
The token `WriteToken` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
!cp "/content/drive/MyDrive/Colab Notebooks/data/index_faiss" index_faiss
!cp "/content/drive/MyDrive/Colab Notebooks/data/boston_housing.csv" boston_housing.csv

# VLLM

```bash
VLLM_BACKEND=FLASHINFER VLLM_USE_V1=1 VLLM_ALLOW_LONG_MAX_MODEL_LEN=1 TOKENIZERS_PARALLELISM=true MAX_JOBS=2 vllm serve ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g --port 8877 --max-model-len 4096 --api-key token-abc123 --quantization compressed-tensors --max-num-seqs=1
```

```bash
VLLM_BACKEND=FLASHINFER VLLM_USE_V1=1 VLLM_ALLOW_LONG_MAX_MODEL_LEN=1 TOKENIZERS_PARALLELISM=true MAX_JOBS=2 vllm serve gaunernst/gemma-3-27b-it-int4-awq --port 8877 --max-model-len 4096 --api-key token-abc123 --quantization awq --dtype half --max-num-seqs=1
```

In [21]:
from openai import OpenAI
import math
import time
import json

client = OpenAI(
    base_url="http://localhost:8877/v1",
    api_key="token-abc123",
)

In [22]:
def generate_response(message_list):
    completion = client.chat.completions.create(
        model = "ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g",
        messages = message_list,
        max_tokens=1024,
        frequency_penalty=0.3,
        temperature=0.6,
        stream=True,
    )

    final_answer = []
    assistant_response = ""

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")
    return assistant_response

In [10]:
message_list = [{"role": "system", "content": "당신은 유저의 질문에 최대한 정확하고 풍부한 정보를 전달하는 assistant 이다. 답변은 항상 한국어로 공손하게 답변해줘."}]

while True:
    user_prompt = input("USER > ")
    if user_prompt.lower() == "quit":
        break
    message_list.append({"role": "user", "content": user_prompt})

    assistant = generate_response(message_list)
    message_list.append({"role": "assistant", "content": assistant})

USER > 헬로
안녕하세요! 무엇을 도와드릴까요? 궁금한 점이 있다면 언제든지 편하게 질문해주세요. 최대한 정확하고 풍부한 정보를 드리도록 노력하겠습니다. 😊


inference time: 2.60750 sec 


USER > quit


In [7]:
system_prompt = "You are a helpful assistant. And Answers must be in Korean."
user_prompt = """#입력
[원본 prompt] = 테니스를 잘 칠 수 있는 방법은?

#처리
당신은 세계 최고의 prompt engineer로 행동합니다.
당신의 역할은 사용자가 제공한 prompt를 높은 품질의 prompt로 개선하는 일 입니다.
다음의 지침에 따라 [원본 prompt]를 재구성하여 [최적화된 prompt] 를 생성해주세요.

1. 의도 파악: 사용자의 궁극적인 목표와 의도를 명확히 이해하고 명시하세요.
2. 맥락 확립: 관련된 배경 정보, 주제의 범위, 필요한 전문 지식 수준을 설정하세요.
3. 구체성 강화: 모호한 표현을 피하고, 명확하고 구체적인 지시사항을 제공하세요.
4. 구조화: 응답의 형식, 구조, 섹션을 명확히 지정하여 체계적인 출력을 유도하세요.
5. 예시 제공: 가능한 경우, 기대하는 출력의 구체적인 예시나 셈플 데이터를 포함하세요.
6. 제한 설정: 응답의 길이, 복잡성 수준, 사용할 기술 용어의 범위 등을 명시하세요.
7. 다각적 접근: 다양한 관점, 시나리오, 또는 해결 방법을 고려하도록 요청하세요.
8. 윤리적 고려: 편함, 민감한 내용, 또는 잠재적인 부작용에 대한 주의사항을 포함하세요.
9. 상호작용 유도: 필요한 경우, 추가 정보나 명확화를 요청할 수 있는 옵션을 제공하세요.
10. 품질 기준 설정: 응답을 평가할 수 있는 구체적인 기준이나 체크리스트를 포함하세요.

#출력
반드시 최적화된 prompt 만 출력합니다.
[최적화된 prompt]

"""

In [8]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
generate_response(messages)

테니스 실력 향상을 위한 구체적인 방법들을 알려주세요. 다음 사항들을 포함하여 답변해주세요:

1. **기초 기술:**
    * 올바른 그립(Eastern, Western, Continental 등)의 종류와 각 그립이 어떤 상황에 적합한지 설명해주세요.
    * 포핸드, 백핸드 스트로크의 정확한 자세와 연습 방법을 단계별로 알려주세요. (비디오 링크나 이미지 포함 가능)
    * 서브의 종류(플랫, 슬라이스, 탑스핀)와 각 서브를 구사하기 위한 자세 및 연습 방법을 설명해주세요.
    * 발리, 스매시, 드롭샷 등 다양한 기술들의 기본 원리와 연습 방법을 알려주세요.

2. **전술 및 전략:**
    * 싱글/더블 경기에서 효과적인 전술은 무엇이며, 어떻게 적용해야 할까요? (예: 네트 플레이, 베이스라인 플레이, 공격/수비 전환)
    * 상대방의 약점을 파악하고 활용하는 전략은 무엇일까요?
    * 코트 위치 선정 및 움직임의 중요성과 연습 방법을 알려주세요.

3. **체력 및 훈련:**
    * 테니스에 필요한 근력, 지구력, 유연성을 향상시키기 위한 운동 루틴을 제시해주세요. (각 운동별 세트 수, 반복 횟수 포함)
    * 경기 중 집중력을 유지하고 심리적 압박을 극복하는 방법을 알려주세요.
    * 부상 예방을 위한 스트레칭 및 워밍업/쿨다운 방법을 설명해주세요.

4. **장비:**
    * 라켓 선택 시 고려해야 할 요소(무게, 헤드 사이즈, 밸런스 등)는 무엇이며, 자신에게 맞는 라켓을 고르는 방법을 알려주세요.
    * 테니스화의 중요성과 선택 기준을 설명해주세요.

5. **추가 정보:**
    * 테니스 실력 향상에 도움이 되는 온라인 자료(웹사이트, 유튜브 채널 등)를 추천해주세요.
    * 초보자가 흔히 저지르는 실수와 이를 개선하는 방법을 알려주세요.

답변은 초보자도 이해하기 쉽도록 상세하게 설명해주시고, 가능한 한 시각 자료(이미지 또는 비디오 링크)를 포함하여 주세요. 또한 각 기술/전략/훈련법의 

'테니스 실력 향상을 위한 구체적인 방법들을 알려주세요. 다음 사항들을 포함하여 답변해주세요:\n\n1. **기초 기술:**\n    * 올바른 그립(Eastern, Western, Continental 등)의 종류와 각 그립이 어떤 상황에 적합한지 설명해주세요.\n    * 포핸드, 백핸드 스트로크의 정확한 자세와 연습 방법을 단계별로 알려주세요. (비디오 링크나 이미지 포함 가능)\n    * 서브의 종류(플랫, 슬라이스, 탑스핀)와 각 서브를 구사하기 위한 자세 및 연습 방법을 설명해주세요.\n    * 발리, 스매시, 드롭샷 등 다양한 기술들의 기본 원리와 연습 방법을 알려주세요.\n\n2. **전술 및 전략:**\n    * 싱글/더블 경기에서 효과적인 전술은 무엇이며, 어떻게 적용해야 할까요? (예: 네트 플레이, 베이스라인 플레이, 공격/수비 전환)\n    * 상대방의 약점을 파악하고 활용하는 전략은 무엇일까요?\n    * 코트 위치 선정 및 움직임의 중요성과 연습 방법을 알려주세요.\n\n3. **체력 및 훈련:**\n    * 테니스에 필요한 근력, 지구력, 유연성을 향상시키기 위한 운동 루틴을 제시해주세요. (각 운동별 세트 수, 반복 횟수 포함)\n    * 경기 중 집중력을 유지하고 심리적 압박을 극복하는 방법을 알려주세요.\n    * 부상 예방을 위한 스트레칭 및 워밍업/쿨다운 방법을 설명해주세요.\n\n4. **장비:**\n    * 라켓 선택 시 고려해야 할 요소(무게, 헤드 사이즈, 밸런스 등)는 무엇이며, 자신에게 맞는 라켓을 고르는 방법을 알려주세요.\n    * 테니스화의 중요성과 선택 기준을 설명해주세요.\n\n5. **추가 정보:**\n    * 테니스 실력 향상에 도움이 되는 온라인 자료(웹사이트, 유튜브 채널 등)를 추천해주세요.\n    * 초보자가 흔히 저지르는 실수와 이를 개선하는 방법을 알려주세요.\n\n답변은 초보자도 이해하기 쉽도록 상세하게 설명해주시고, 가능한 한 시각 자료(이미지 또는 비디오 링크)를

In [9]:
system_prompt = "You are a helpful assistant. And Answers must be in Korean."
user_prompt = """프로 테니스 대회에서 테니스 공은 한번에 6개를 사용합니다. 이 6개의 공을 처음에는 게임 수의 합이 7게임, 다음부터는 9게임마다 새 공으로 교체를 합니다.
만일 3세트 경기가 6:5 3:6 6:4 로 진행됐다고 하면 총 몇 개의 공을 사용했을까요?
답:
각 세트마다 게임 수를 더하면 11+9+10 = 30 으로 총 30게임이 진행됐습니다.
테니스 공은 7번째 교체 후 9번째 게임마다 교체되니 7,16,25 게임에 총 3회에 교체 됩니다.
최종적으로 경기시작 시 사용한 공 6개 + 교체 시 마다 6개의 새 공으로 교체 했으니 6 + (6 * 3) = 24, 사용된 공은 총 24개 입니다.

질문:
아마추어 테니스 대회에서는 테니스공을 한번에 2개 사용합니다. 그리고 이 2개의 공을 처음에는 게임 수의 합이 7게임, 다음부터는 9게임마다 새공으로 교체를 합니다.
만일 3세트 경기가 6:5 5:7 6:7 로 진행됐다고 하면 총 몇 개의 공을 사용했을까요?"""

In [10]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
generate_response(messages)

답:

각 세트마다 게임 수를 더하면 11 + 12 + 13 = 36 으로 총 36게임이 진행됐습니다.
테니스 공은 7번째 게임 후, 그리고 그 이후 9게임마다 교체되므로, 7, 16, 25, 34 게임에 총 4회 교체됩니다.
경기 시작 시 사용한 공 2개와 교체 시마다 추가되는 공 2개를 합하면, 2 + (2 * 4) = 10 개입니다.
따라서 사용된 공은 총 10개입니다.

inference time: 10.22014 sec 




'답:\n\n각 세트마다 게임 수를 더하면 11 + 12 + 13 = 36 으로 총 36게임이 진행됐습니다.\n테니스 공은 7번째 게임 후, 그리고 그 이후 9게임마다 교체되므로, 7, 16, 25, 34 게임에 총 4회 교체됩니다.\n경기 시작 시 사용한 공 2개와 교체 시마다 추가되는 공 2개를 합하면, 2 + (2 * 4) = 10 개입니다.\n따라서 사용된 공은 총 10개입니다.'

In [11]:
user_prompt = """아래의 영화 줄거리를 기반으로 해당 영화가 어떤 장르인지 설명해줘.
영화 줄거리:
세계 최고 바둑 대회에서 국내 최초 우승자가 된 조훈현. 전 국민적 영웅으로 대접받던 그는 바둑 신동이라 불리는 이창호를 제자로 맞는다. “실전에선 기세가 8할이야” 제자와 한 지붕 아래에서 먹고 자며 가르친 지 수년. 모두가 스승의 뻔한 승리를 예상했던 첫 사제 대결에서 조훈현은 전 국민이 지켜보는 가운데, 기세를 탄 제자에게 충격적으로 패한다. 오랜만에 패배를 맛본 조훈현과 이제 승부의 맛을 알게 된 이창호. 조훈현은 타고난 승부사적 기질을 되살리며 다시 한번 올라갈 결심을 하게 되는데"""

In [12]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
generate_response(messages)

이 영화 줄거리는 **드라마** 장르에 속한다고 볼 수 있습니다. 

몇 가지 이유를 들어 설명하자면:

*   **인물들의 내면 갈등:** 조훈현과 이창호, 두 인물의 심리 변화와 성장을 다루고 있습니다. 패배 후 조훈현이 다시 일어서기로 결심하는 과정은 드라마에서 자주 나타나는 전개입니다.
*   **관계의 변화:** 스승과 제자 사이의 복잡한 관계, 그리고 경쟁을 통해 변화하는 관계를 중심으로 이야기가 전개됩니다.
*   **감정적인 요소:** 전 국민적 영웅의 패배, 제자의 성장 등 감정을 자극하는 요소들이 포함되어 있습니다.
*   **실화 기반:** 실제 바둑 기사들의 이야기를 바탕으로 하고 있을 가능성이 높으며, 실화를 바탕으로 한 이야기는 드라마 장르에서 흔히 찾아볼 수 있습니다.

물론 바둑이라는 소재를 사용하고 있기에 스포츠 영화의 요소도 있지만, 주된 초점은 인물들의 감정 변화와 관계에 있기 때문에 드라마 장르로 분류하는 것이 더 적절하다고 생각합니다.

inference time: 17.91799 sec 




'이 영화 줄거리는 **드라마** 장르에 속한다고 볼 수 있습니다. \n\n몇 가지 이유를 들어 설명하자면:\n\n*   **인물들의 내면 갈등:** 조훈현과 이창호, 두 인물의 심리 변화와 성장을 다루고 있습니다. 패배 후 조훈현이 다시 일어서기로 결심하는 과정은 드라마에서 자주 나타나는 전개입니다.\n*   **관계의 변화:** 스승과 제자 사이의 복잡한 관계, 그리고 경쟁을 통해 변화하는 관계를 중심으로 이야기가 전개됩니다.\n*   **감정적인 요소:** 전 국민적 영웅의 패배, 제자의 성장 등 감정을 자극하는 요소들이 포함되어 있습니다.\n*   **실화 기반:** 실제 바둑 기사들의 이야기를 바탕으로 하고 있을 가능성이 높으며, 실화를 바탕으로 한 이야기는 드라마 장르에서 흔히 찾아볼 수 있습니다.\n\n물론 바둑이라는 소재를 사용하고 있기에 스포츠 영화의 요소도 있지만, 주된 초점은 인물들의 감정 변화와 관계에 있기 때문에 드라마 장르로 분류하는 것이 더 적절하다고 생각합니다.'

In [13]:
user_prompt = "python 에서 list 에 값 추가하는 코드 생성해줘."

In [14]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
generate_response(messages)

```python
# 리스트에 값 추가하는 방법은 여러 가지가 있습니다.

# 1. append() 메서드: 리스트의 맨 뒤에 값 하나를 추가합니다.
my_list = [1, 2, 3]
my_list.append(4)
print(my_list)  # 출력: [1, 2, 3, 4]

# 2. insert() 메서드: 특정 인덱스 위치에 값을 추가합니다.
my_list = [1, 2, 3]
my_list.insert(1, 5)  # 인덱스 1 위치에 5를 추가합니다.
print(my_list)  # 출력: [1, 5, 2, 3]

# 3. extend() 메서드: 다른 리스트의 모든 요소를 리스트의 맨 뒤에 추가합니다.
my_list = [1, 2, 3]
another_list = [4, 5, 6]
my_list.extend(another_list)
print(my_list)  # 출력: [1, 2, 3, 4, 5, 6]

# 4. + 연산자: 두 리스트를 연결하여 새로운 리스트를 생성합니다.
my_list = [1, 2, 3]
another_list = [4, 5, 6]
new_list = my_list + another_list
print(new_list)  # 출력: [1, 2, 3, 4, 5, 6]

# 주의사항: append() 와 insert() 는 리스트 자체를 변경하지만 + 연산자는 새로운 리스트를 생성합니다.
```

**설명:**

*   **`append()`**: 가장 일반적으로 사용되는 방법으로 리스트 끝에 요소를 추가합니다.
*   **`insert()`**: 특정 위치에 요소를 삽입해야 할 때 유용합니다. 인덱스와 값을 인자로 받습니다.
*   **`extend()`**: 다른 리스트의 모든 요소를 한 번에 추가할 때 사용합니다.  리스트를 확장하는 데 적합합니다.
*   **`+` 연산자**: 두 리스트를 연결하여 새로운 리스트를 만들 때 사용합니다. 기존 리스트는 변경되지 않습니다.

어떤 방법을 사용할지는 상황에 따라 다릅니다.  

'```python\n# 리스트에 값 추가하는 방법은 여러 가지가 있습니다.\n\n# 1. append() 메서드: 리스트의 맨 뒤에 값 하나를 추가합니다.\nmy_list = [1, 2, 3]\nmy_list.append(4)\nprint(my_list)  # 출력: [1, 2, 3, 4]\n\n# 2. insert() 메서드: 특정 인덱스 위치에 값을 추가합니다.\nmy_list = [1, 2, 3]\nmy_list.insert(1, 5)  # 인덱스 1 위치에 5를 추가합니다.\nprint(my_list)  # 출력: [1, 5, 2, 3]\n\n# 3. extend() 메서드: 다른 리스트의 모든 요소를 리스트의 맨 뒤에 추가합니다.\nmy_list = [1, 2, 3]\nanother_list = [4, 5, 6]\nmy_list.extend(another_list)\nprint(my_list)  # 출력: [1, 2, 3, 4, 5, 6]\n\n# 4. + 연산자: 두 리스트를 연결하여 새로운 리스트를 생성합니다.\nmy_list = [1, 2, 3]\nanother_list = [4, 5, 6]\nnew_list = my_list + another_list\nprint(new_list)  # 출력: [1, 2, 3, 4, 5, 6]\n\n# 주의사항: append() 와 insert() 는 리스트 자체를 변경하지만 + 연산자는 새로운 리스트를 생성합니다.\n```\n\n**설명:**\n\n*   **`append()`**: 가장 일반적으로 사용되는 방법으로 리스트 끝에 요소를 추가합니다.\n*   **`insert()`**: 특정 위치에 요소를 삽입해야 할 때 유용합니다. 인덱스와 값을 인자로 받습니다.\n*   **`extend()`**: 다른 리스트의 모든 요소를 한 번에 추가할 때 사용합니다.  리스트를 확장하는 데 적합합니다.\n*   **`+` 연산자**: 두 리스트를 연결하여 새로운 리스트를 만들 때 사용합니다. 기존 리스트는 변경되지 

In [20]:
from openai import OpenAI
import math
import time
import json

client = OpenAI(
    base_url="http://localhost:8877/v1",
    api_key="token-abc123",
)

In [21]:
def generate_response(message_list):
    completion = client.chat.completions.create(
        model = "ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g",
        messages = message_list,
        max_tokens=1024,
        frequency_penalty=0.3,
        temperature=0.6,
        stream=True,
    )

    final_answer = []
    assistant_response = ""

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")
    return assistant_response

In [22]:
system_prompt = "You are a helpful assistant. And Answers must be in Korean."
user_prompt = "(x^5-12x^3+9x)/(x^3-4x) 그래프 그려줘."

In [25]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
assistant_response = generate_response(messages)

알겠습니다. (x^5-12x^3+9x)/(x^3-4x) 그래프를 그려 드리겠습니다.

**함수 분석:**

먼저 함수를 간단히 정리해 보겠습니다.

f(x) = (x^5 - 12x^3 + 9x) / (x^3 - 4x) = (x(x^4 - 12x^2 + 9)) / (x(x^2 - 4)) = (x^4 - 12x^2 + 9) / (x^2 - 4)  (단, x ≠ 0)

분자와 분모를 x^2-4로 나누면 다음과 같습니다.

f(x) = x^2 - 8 + (-23)/(x^2-4)

**그래프 특징:**

*   **근:** 함수가 x축과 만나는 점은 근입니다. 분자가 0이 되는 지점을 찾으면 됩니다. x^4 - 12x^2 + 9 = 0 을 풀면 근을 구할 수 있습니다.
*   **극값:** 도함수를 구하여 극대/극소를 찾을 수 있습니다.
*   **점근선:** 분모가 0이 되는 지점에서 수직 점근선이 존재합니다. 즉, x = 2와 x = -2에서 수직 점근선이 있습니다. 또한, x가 무한대로 갈 때 함수의 극한값을 구하여 수평 점근선을 찾을 수 있습니다.
*   **대칭성:** 함수가 짝함수인지 홀함수인지 확인합니다. f(-x) = f(x) 이면 짝함수, f(-x) = -f(x) 이면 홀함수입니다. 이 함수는 짝함수가 아닙니다.
* **y절편**: x=0일 때 y값입니다. 하지만 원래 함수에서 x=0이 정의되지 않으므로 y절편은 없습니다.

**그래프 개략적인 형태:**

1.  **수직 점근선:** x = -2 및 x = 2 에서 수직 점근선을 가집니다.
2.  **극값:** 그래프는 극대값과 극소값을 가집니다 (도함수를 통해 정확한 위치를 찾을 수 있습니다).
3.  **포물선 형태:** 큰 값의 x에 대해 f(x)는 대략적으로 x^2에 가까워지므로 포물선 형태를 가집니다.
4.  **불연속점**: x=0에서 불연속점을 가집니다.(원래 함수에서 제외했습니다.)

**그래프 이미지:**

[https://www.wolframalpha.com/input?i=%28x%5

In [27]:
system_prompt = """<start_of_turn>system

Environment: ipython
Tools: brave_search, wolfram_alpha

Cutting Knowledge Date: December 2023
Today Date: 05 Apr 2025
You are a helpful Assistant.<end_of_turn>"""

user_prompt = "What is the current weather in Menlo Park, California?"
#user_prompt = "Can you help me solve this equation: x^3 - 4x^2 + 6x - 24 = 0"

In [30]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
assistant_response = generate_response(messages)

```json
{"name": "get_current_conditions", "parameters": {"location": "San Francisco, CA", "unit": "Fahrenheit"}}
```

inference time: 2.85902 sec 




In [31]:
system_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful assistant with tool calling capabilities. When you receive a tool call response, use the output to format an answer to the orginal use question.<|eot_id|><|start_header_id|>user<|end_header_id|>
Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.
Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.
{
    "type": "function",
    "function": {
    "name": "get_current_conditions",
    "description": "Get the current weather conditions for a specific location",
    "parameters": {
        "type": "object",
        "properties": {
        "location": {
            "type": "string",
            "description": "The city and state, e.g., San Francisco, CA"
        },
        "unit": {
            "type": "string",
            "enum": ["Celsius", "Fahrenheit"],
            "description": "The temperature unit to use. Infer this from the user's location."
        }
        },
        "required": ["location", "unit"]
    }
    }
}
"""

user_prompt = "what is the weather like in San Fransisco?"

In [32]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
assistant_response = generate_response(messages)

```json
{"name": "get_current_conditions", "parameters": {"location": "San Francisco, CA", "unit": "Fahrenheit"}}
```

inference time: 2.50977 sec 




In [50]:
system_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are a helpful assistant with tool calling capabilities. When you receive a tool call response, use the output to format an answer to the orginal use question.<|eot_id|><|start_header_id|>user<|end_header_id|>
Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.
Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.
{
    "type": "function",
    "function": {
        "name": "execute_python",
        "description": "Execute python code in a Jupyter notebook cell and returns any result, stdout, stderr, display_data, and error.",
        "parameters": {
              "type": "object",
              "properties": {
                  "code": {
                    "type": "string",
                    "description": "The python code to execute in a single cell.",
                  }
              },
              "required": ["code"],
          },
      },
}
"""

user_prompt = """boston_housing.csv 데이터 EDA 를 python 코드 활용해서 수행하고 결과를 분석해줘. boston_housing.csv 파일은 /home/freenak/kernel_gate/ 디렉토리에 있으니 꼭 해당 위치의 파일을 로드해야해.
"""

In [51]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
assistant_response = generate_response(messages)

```json
{
  "name": "execute_python",
  "parameters": {
    "code": "import pandas as pd\n\n# 데이터 로드\ndata = pd.read_csv('/home/freenak/kernel_gate/boston_housing.csv')\n\n# 데이터 정보 확인\nprint(\"데이터 정보:\")\nprint(data.info())\n\n# 기술 통계량 확인\nprint(\"\\n기술 통계량:\")\nprint(data.describe())\n\n# 결측치 확인\nprint(\"\\n결측치:\")\nprint(data.isnull().sum())\n\n# 상관 관계 분석\nprint(\"\\n상관 관계:\")\nprint(data.corr())\n\n# 주요 변수 분포 확인 (예시)\nimport matplotlib.pyplot as plt\nplt.figure(figsize=(12, 6))\nsample_columns = ['CRIM', 'RM', 'PTRATIO', 'MEDV'] # 예시로 4개 변수 선택\nfor i, column in enumerate(sample_columns):\n    plt.subplot(2, 2, i + 1)\n    data[column].hist()\n    plt.title(column)\nplt.tight_layout()\nplt.show()\n\n# 분석 결과 요약 (예시)\nprint(\"\\n분석 결과 요약:\")\nprint(\"데이터는 총 {}개의 행과 {}개의 열을 가지고 있습니다.\".format(data.shape[0], data.shape[1]))\nprint(\"범죄율(CRIM)과 주택 가격(MEDV)은 음의 상관 관계를 가집니다.\")\nprint(\"방의 개수(RM)와 주택 가격(MEDV)은 양의 상관 관계를 가집니다.\")"
  }
}
```

inference time: 31.07349 sec 




In [61]:
import json
import pandas as pd

import time
import requests
import base64
from IPython.display import display, Image
from importlib.metadata import PackageNotFoundError, distribution
from uuid import uuid4
from websockets.sync.client import connect as ws_connect_sync

%matplotlib inline

ws_url = "127.0.0.1:8892/api"

In [62]:
def kg_connect(kernel_url):
    response = requests.post(
        f"http://{kernel_url}/kernels",
        headers={"Content-Type": "application/json"},
        timeout=270,
    )
    kernel_id = response.json()["id"]
    print(f'kernel_id: {kernel_id}')

    if kernel_id is None:
        raise Exception("Could not start kernel")

    ws = ws_connect_sync(f"ws://{kernel_url}/kernels/{kernel_id}/channels")
    return ws

In [63]:
def exec_python(python_code):
    jupyter_webSocket = kg_connect(ws_url)

    send_msg = json.dumps(
        {
            "header": {
                "msg_id": (msg_id := uuid4().hex),
                "msg_type": "execute_request",
            },
            "parent_header": {},
            "metadata": {},
            "content": {
                "code": python_code,
                "silent": False,
                "store_history": True,
                "user_expressions": {},
                "allow_stdin": False,
                "stop_on_error": True,
            },
            "channel": "shell",
            "buffers": [],
        }
    )

    print(f'LLM generate code: \n{python_code}')

    jupyter_webSocket.send(send_msg)
    return_string = ''

    while True:
        received_msg = json.loads(jupyter_webSocket.recv())

        if ( received_msg["header"]["msg_type"] == "stream"
            and received_msg["parent_header"]["msg_id"] == msg_id ):
            msg = received_msg["content"]["text"].strip()
            if "Requirement already satisfied:" in msg:
                continue
            result = msg + "\n"
            return_string += result
            print("Output:\n", result)

        elif ( received_msg["header"]["msg_type"] == "execute_result"
              and received_msg["parent_header"]["msg_id"] == msg_id ):
            result = received_msg["content"]["data"]["text/plain"].strip() + "\n"
            return_string += result
            print("Output:\n", result)

        elif received_msg["header"]["msg_type"] == "display_data":
            if "image/png" in received_msg["content"]["data"]:
                result = received_msg["content"]["data"]["image/png"]
                return_string += "The graph/picture representation on the user screen has been completed."

                img_bytes = base64.b64decode(result)
                display(Image(img_bytes))

            if "text/plain" in received_msg["content"]["data"]:
                result = received_msg["content"]["data"]["text/plain"]
                return_string += result
                print("Output:\n", result)

        elif received_msg["header"]["msg_type"] == "execute_reply":
            if received_msg["content"]["status"] == "ok":
                print("Output: end of received_msg\n" )
                jupyter_webSocket.close()
                break

        elif received_msg["header"]["msg_type"] == "error":
            result = received_msg["content"]["evalue"]
            print("Output: Error\n", result)
            jupyter_webSocket.close()
            break

        else :
            time.sleep(0.2)

    return return_string

In [52]:
assistant_response

'```json\n{\n  "name": "execute_python",\n  "parameters": {\n    "code": "import pandas as pd\\n\\n# 데이터 로드\\ndata = pd.read_csv(\'/home/freenak/kernel_gate/boston_housing.csv\')\\n\\n# 데이터 정보 확인\\nprint(\\"데이터 정보:\\")\\nprint(data.info())\\n\\n# 기술 통계량 확인\\nprint(\\"\\\\n기술 통계량:\\")\\nprint(data.describe())\\n\\n# 결측치 확인\\nprint(\\"\\\\n결측치:\\")\\nprint(data.isnull().sum())\\n\\n# 상관 관계 분석\\nprint(\\"\\\\n상관 관계:\\")\\nprint(data.corr())\\n\\n# 주요 변수 분포 확인 (예시)\\nimport matplotlib.pyplot as plt\\nplt.figure(figsize=(12, 6))\\nsample_columns = [\'CRIM\', \'RM\', \'PTRATIO\', \'MEDV\'] # 예시로 4개 변수 선택\\nfor i, column in enumerate(sample_columns):\\n    plt.subplot(2, 2, i + 1)\\n    data[column].hist()\\n    plt.title(column)\\nplt.tight_layout()\\nplt.show()\\n\\n# 분석 결과 요약 (예시)\\nprint(\\"\\\\n분석 결과 요약:\\")\\nprint(\\"데이터는 총 {}개의 행과 {}개의 열을 가지고 있습니다.\\".format(data.shape[0], data.shape[1]))\\nprint(\\"범죄율(CRIM)과 주택 가격(MEDV)은 음의 상관 관계를 가집니다.\\")\\nprint(\\"방의 개수(RM)와 주택 가격(MEDV)은 양의 상관 관

In [58]:
jsson_return = json.loads(assistant_response.replace("```json", "").replace("```", ""))
jsson_return['parameters']['code']

'import pandas as pd\n\n# 데이터 로드\ndata = pd.read_csv(\'/home/freenak/kernel_gate/boston_housing.csv\')\n\n# 데이터 정보 확인\nprint("데이터 정보:")\nprint(data.info())\n\n# 기술 통계량 확인\nprint("\\n기술 통계량:")\nprint(data.describe())\n\n# 결측치 확인\nprint("\\n결측치:")\nprint(data.isnull().sum())\n\n# 상관 관계 분석\nprint("\\n상관 관계:")\nprint(data.corr())\n\n# 주요 변수 분포 확인 (예시)\nimport matplotlib.pyplot as plt\nplt.figure(figsize=(12, 6))\nsample_columns = [\'CRIM\', \'RM\', \'PTRATIO\', \'MEDV\'] # 예시로 4개 변수 선택\nfor i, column in enumerate(sample_columns):\n    plt.subplot(2, 2, i + 1)\n    data[column].hist()\n    plt.title(column)\nplt.tight_layout()\nplt.show()\n\n# 분석 결과 요약 (예시)\nprint("\\n분석 결과 요약:")\nprint("데이터는 총 {}개의 행과 {}개의 열을 가지고 있습니다.".format(data.shape[0], data.shape[1]))\nprint("범죄율(CRIM)과 주택 가격(MEDV)은 음의 상관 관계를 가집니다.")\nprint("방의 개수(RM)와 주택 가격(MEDV)은 양의 상관 관계를 가집니다.")'

In [95]:
python_code = jsson_return['parameters']['code']
python_code

'import pandas as pd\n\n# 데이터 로드\ndata = pd.read_csv(\'/home/freenak/kernel_gate/boston_housing.csv\')\n\n# 데이터 정보 확인\nprint("데이터 정보:")\nprint(data.info())\n\n# 기술 통계량 확인\nprint("\\n기술 통계량:")\nprint(data.describe())\n\n# 결측치 확인\nprint("\\n결측치:")\nprint(data.isnull().sum())\n\n# 상관 관계 분석\nprint("\\n상관 관계:")\nprint(data.corr())\n\n# 주요 변수 분포 확인 (예시)\nimport matplotlib.pyplot as plt\nplt.figure(figsize=(12, 6))\nsample_columns = [\'CRIM\', \'RM\', \'PTRATIO\', \'MEDV\'] # 예시로 4개 변수 선택\nfor i, column in enumerate(sample_columns):\n    plt.subplot(2, 2, i + 1)\n    data[column].hist()\n    plt.title(column)\nplt.tight_layout()\nplt.show()\n\n# 분석 결과 요약 (예시)\nprint("\\n분석 결과 요약:")\nprint("데이터는 총 {}개의 행과 {}개의 열을 가지고 있습니다.".format(data.shape[0], data.shape[1]))\nprint("범죄율(CRIM)과 주택 가격(MEDV)은 음의 상관 관계를 가집니다.")\nprint("방의 개수(RM)와 주택 가격(MEDV)은 양의 상관 관계를 가집니다.")'

In [96]:
from IPython.utils import traitlets as _traitlets

exec_result = exec_python(python_code)

kernel_id: 74c9f86b-f966-4eed-b768-d1bb241f3a3f


TimeoutError: timed out while waiting for handshake response

In [ ]:
 print(exec_result)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    int64  
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  PRICE    506 non-null    float64
dtypes: float64(11), int64(3)
memory usage: 55.5 KB
None
             CRIM          ZN       INDUS        CHAS         NOX          RM  \
count  506.000000  506.000000  506.000000  506.000000  506.000000  506.000000   
mean     3.613524   11.363636   11.136779    0.069170    0.554695   

In [ ]:
messages.append({"role": "assistant", "content": assistant_response})
messages.append({"role": "user", "content": exec_result})
assistant_response = generate_response(messages)

boston_housing.csv의 EDA 결과는 다음과 같이 요약할 수 있습니다.

1. **데이터 개요**
   - 총 506개의 데이터가 있으며, 14개의 특성과 1개의 타겟 변수가 있습니다.
   - 특성의 데이터 타입은 float64와 int64로 구성되어 있습니다.

2. **특성 통계**
   - CRIM: 범죄율, 평균 3.61, 표준편차 8.60
   - ZN: 지구 내 비건물 면적, 평균 11.36, 표준편차 23.32
   - INDUS: 상업/업무용지 비율, 평균 11.14, 표준편차 6.86
   - CHAS: Charles River와의 거리 (0: river = 0, no river = 1), 평균 0.07
   - NOX: 일산화질소 배출량, 평균 0.55, 표준편차 0.12
   - RM: 집당 방의 수, 평균 6.28, 표준편차 0.70
   - AGE: 집의 연령 (1940년), 평균 68.57, 표준편차 28.15
   - DIS: 근처의 중간 학교 및 고등학교까지의 거리, 평균 3.80, 표준편차 2.11
   - RAD: 교통 시설 (1-8), 평균 9.55, 표준편차 8.71
   - TAX: 집당 세금 (10^-2), 평균 408.24, 표준편차 168.54
   - PTRATIO: 학생과 교사 비율 (1000), 평균 18.46, 표준편차 2.16
   - B: 집당 인구 수 (1000), 평균 356.67, 표준편차 91.29
   - LSTAT: 인구의 중위 소득 (1000), 평균 12.65, 표준편차 7.14

3. **타겟 변수 통계**
   - PRICE(가격): $22,532

4. **모델링**
   랜덤 포레스트 모델을 사용하여 학습/테스트 세트를 분리하고 예측을 수행했습니다.
   MSE는 약 $9,\!555$입니다.

이 결과는 boston_housing.csv 데이터에 대한 간단한 EDA를 제공하며 모델링 성능을 평가하는 데 사용할 수 있습니다.

{"name": "execute_python",

In [85]:
system_prompt = "You are a helpful Code assistant. Expert in Python. Answer only with  Codes."
user_prompt = "manifold 문제 해결하는 코드 짜서 pandas 로 시각화 해줘. 함수실행도 포함해서"

In [88]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
assistant_response = generate_response(messages)
python_code = assistant_response.replace("```python\n", "").replace("```", "")

In [93]:
with open('python_code.py', 'w') as f:
    f.write(python_code)

  File "/content/python_code.py", line 58
    return df
    ^
IndentationError: expected an indented block after 'elif' statement on line 55


In [94]:
!python python_code.py

In [ ]:
messages

# RAG

In [9]:
!pip install --upgrade numpy==2.0.0 --no-deps

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 99.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4


In [10]:
from transformers import AutoTokenizer, AutoModel
import torch, os, faiss
from FlagEmbedding import FlagModel
import numpy as np

In [11]:
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")
raw_model = AutoModel.from_pretrained("BAAI/bge-m3")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [12]:
raw_model.eval()

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(8194, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elem

In [13]:
import datasets

wikidata = datasets.load_dataset('yongchanskii/wiki-korean')
wikidata = wikidata.remove_columns(['id', 'url'])

#embedding_data = wikidata['train'].select(range(1000))

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/381 [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/400M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/205M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/177M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/647897 [00:00<?, ? examples/s]

In [14]:
input_text = wikidata['train']['text'][5490]
input_text

"2011년은 토요일로 시작하는 평년이다.\n\n사건\n\n1월 ~ 6월 \n 1월 1일\n 이집트 알렉산드리아의 알 키디신 콥트교 성당에서 폭탄테러가 발생해 21명이 숨지고, 97명이 부상을 당했다.\n 브라질 첫 여성 대통령 지우마 호세프 대통령이 취임.\n 에스토니아, 유로 도입.\n 1월 3일 - 교황 베네딕토 16세가 1일에 있었던 이집트에서 발생한 폭탄 테러를 강하게 규탄했다.\n 1월 4일\n 파키스탄의 정치인 살만 타시르가 자신의 경호원에게 살해당했다.\n 이란의 마지막 샤인 모하마드 레자 팔라비의 막내 아들 알리 레자 팔라비가 미국에서 권총으로 자살했다.\n 1월 6일 - 아시아나항공 A380 6대 도입 계약 체결.\n 1월 11일\n 아랍에미리트(UAE) 특수전부대의 교육훈련을 지원하고자 대한민국에서 UAE 군사훈련협력단(아크 부대)을 파견하였다.\n 브라질의 리우 데 자네이루 주에서 홍수와 산사태가 일어나 800여 명의 사상자를 냈다.\n 1월 12일 - 아프리카 수단 남부 주민들을 대상으로 분리 독립 투표를 실시함.\n 1월 14일 - 튀니지의 반정부 시위로 인하여 벤 알리 대통령이 축출되었다.\n 1월 21일 - 소말리아 인근 해역에서 대한민국의 선박 삼호 주얼리호 피랍 사건이 발생하였다.\n 1월 24일 - 러시아의 도모데도보 국제공항에서 폭탄 테러가 일어났다.\n 1월 27일 - 강원도의 이광재 지사와 국회의원인 서갑원이 대법원에서 징역형이 확정되어 도지사와 국회의원 직이 상실되었다.\n 2월 2일 - 조선민주주의인민공화국이 대한민국 백령도 인근에 고속 상륙침투가 가능한 새 공기부양정 기지를 건설 중인 것으로 밝혀졌다.\n 2월 4일 - IPv4 주소가 모두 소진되어 할당이 중단되었다.\n 2월 5일 - 조선민주주의인민공화국 주민 31명이 어선을 타고 연평도 동북쪽 북방한계선을 넘어와 귀순 의사를 밝힌 것으로 알려졌다.\n 2월 7일 - 교황 베네딕토 16세가 이성효(리노)신부를 천주교 수원교구 보좌주교로 임명했다.\n 2월 8일 -

In [15]:
length = len(input_text)
length

23361

In [16]:
embedding_text = wikidata['train'].select(range(5500,6000))

In [17]:
embedding_text

Dataset({
    features: ['title', 'text'],
    num_rows: 500
})

In [18]:
embedding_text['text'][190]

'석기 시대(石器時代, )는 인류 역사를 추적하는 데 고고학 및 역사학에서 쓰이는 낱말로서 인류, 정확히는 인류의 조상인 원인이 돌로 만든 도구를 쓰기 시작한 시대를 일컫는다.\n\n석기는 다양한 종류의 돌로 만든다. 부싯돌과 규질암은 자르는 도구나 무기로 만들어졌고, 현무암이나 사암은 맷돌과 같이 가는 도구로 만들어졌다. 나무, 뼈, 조개껍질, 사슴의 뿔, 그리고 다른 재료들도 필요에 따라 만들어 쓴 것으로 보인다. 이 시기의 말기에는 진흙과 같은 점토도 도기를 만들기 위해 사용되었다. 일련의 기술적 진보가 이후, 동기 시대(Chalcolithic), 청동기 시대, 철기 시대처럼 특징화 된다.\n\n최초의 석기는 에티오피아의 고나에서 발견된 270만년에서 258만년 정도된 것이며, 그 이후로 넓게 확산되었다. 이것은 농경, 동물의 가축화, 그리고 금속을 생산하기 위해 구리 광물을 녹이는 기술이 개발된 이후 끝이 나게 된다. 이 시기는 인류가 아직 문자를 만들지 못했기 때문에 선사 시대라고 표현을 한다.\n\n석기 시대라는 용어는 석기가 다른 재료로 만든 것보다 우세한 시기인 야금술 이전의 오랜 기간을 표현하기 위해 고고학자들에 의해 사용되었다.\n\n때문에 도구를 만들어 쓰는 동물, 즉 호모 하빌리스로서 인간은 기원전 약 2백만 년을 전후로 여러 대륙에서 출현한 것으로 여겨지며 기원전 약 9000-8000년에 석기시대는 끝을 맺은 것으로 고고학계에서는 추측하고 있다.\n\n고고학에서 석기 시대 \n\n석기 시대의 시대 범위는 논의가 되는 지역에 따라 모호하며, 이견이 많고, 다양하다. 전체 인류의 석기 시대를 말하는 것은 가능하지만, 어떤 그룹은 금속 제련 기술을 개발하지도 못해 기술적으로 발달된 문화를 접할 때까지 여전히 석기 시대로 남아 있었다. 그러나 석기 시대는 250만년 전에 최초의 고대 인류가 아프리카에서 도구를 만들며 시작되었으며, 오스트랄로피테쿠스 가르히일 가능성이 높다.\n\n아직까지 남아 있는 석기는 이 시기에 고고학적 조사의 주요하면서, 

In [19]:
# get the BGE embedding model
model = FlagModel('BAAI/bge-m3',
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                  use_fp16=True)

In [20]:
# get the embedding of the corpus
wiki_embeddings = model.encode(embedding_text['text'])

print("shape of the corpus embeddings:", wiki_embeddings.shape)
print("data type of the embeddings: ", wiki_embeddings.dtype)

pre tokenize: 100%|██████████| 2/2 [00:00<00:00,  3.12it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings:   0%|          | 0/4 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 576.00 MiB. GPU 0 has a total capacity of 22.16 GiB of which 262.94 MiB is free. Process 41356 has 18.11 GiB memory in use. Process 13728 has 3.79 GiB memory in use. Of the allocated memory 2.05 GiB is allocated by PyTorch, and 1.51 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
wiki_embeddings_np = wiki_embeddings.astype(np.float32)
dim = wiki_embeddings_np.shape[-1]

# create the faiss index and store the corpus embeddings into the vector space
index = faiss.index_factory(dim, 'Flat', faiss.METRIC_INNER_PRODUCT)

In [ ]:
# check if the index is trained
print(index.is_trained)
# index.train(corpus_embeddings)

# add all the vectors to the index
index.add(wiki_embeddings_np)

print(f"total number of vectors: {index.ntotal}")

True
total number of vectors: 500


In [ ]:
queries = [
    "석기시대란 무었인가?",
]

query_embeddings = model.encode_queries(queries)
dists, ids = index.search(query_embeddings.astype(np.float32), k=3)

print(f"query:\t{queries[0]}\n\n")
count = 0
for i in range(3):
    print(f"RAG_{count}:\t{embedding_text['text'][ids[0][i]]}\n")
    count = count+1

query:	석기시대란 무었인가?


RAG_0:	석기 시대(石器時代, )는 인류 역사를 추적하는 데 고고학 및 역사학에서 쓰이는 낱말로서 인류, 정확히는 인류의 조상인 원인이 돌로 만든 도구를 쓰기 시작한 시대를 일컫는다.

석기는 다양한 종류의 돌로 만든다. 부싯돌과 규질암은 자르는 도구나 무기로 만들어졌고, 현무암이나 사암은 맷돌과 같이 가는 도구로 만들어졌다. 나무, 뼈, 조개껍질, 사슴의 뿔, 그리고 다른 재료들도 필요에 따라 만들어 쓴 것으로 보인다. 이 시기의 말기에는 진흙과 같은 점토도 도기를 만들기 위해 사용되었다. 일련의 기술적 진보가 이후, 동기 시대(Chalcolithic), 청동기 시대, 철기 시대처럼 특징화 된다.

최초의 석기는 에티오피아의 고나에서 발견된 270만년에서 258만년 정도된 것이며, 그 이후로 넓게 확산되었다. 이것은 농경, 동물의 가축화, 그리고 금속을 생산하기 위해 구리 광물을 녹이는 기술이 개발된 이후 끝이 나게 된다. 이 시기는 인류가 아직 문자를 만들지 못했기 때문에 선사 시대라고 표현을 한다.

석기 시대라는 용어는 석기가 다른 재료로 만든 것보다 우세한 시기인 야금술 이전의 오랜 기간을 표현하기 위해 고고학자들에 의해 사용되었다.

때문에 도구를 만들어 쓰는 동물, 즉 호모 하빌리스로서 인간은 기원전 약 2백만 년을 전후로 여러 대륙에서 출현한 것으로 여겨지며 기원전 약 9000-8000년에 석기시대는 끝을 맺은 것으로 고고학계에서는 추측하고 있다.

고고학에서 석기 시대 

석기 시대의 시대 범위는 논의가 되는 지역에 따라 모호하며, 이견이 많고, 다양하다. 전체 인류의 석기 시대를 말하는 것은 가능하지만, 어떤 그룹은 금속 제련 기술을 개발하지도 못해 기술적으로 발달된 문화를 접할 때까지 여전히 석기 시대로 남아 있었다. 그러나 석기 시대는 250만년 전에 최초의 고대 인류가 아프리카에서 도구를 만들며 시작되었으며, 오스트랄로피테쿠스 가르히일 가능성이 높다.

아직까지 남아 있는 석기는 이 시기에 고고학

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

In [ ]:
texts = text_splitter.create_documents(embedding_text['text'])

Created a chunk of size 666, which is longer than the specified 500
Created a chunk of size 699, which is longer than the specified 500
Created a chunk of size 561, which is longer than the specified 500
Created a chunk of size 764, which is longer than the specified 500
Created a chunk of size 835, which is longer than the specified 500
Created a chunk of size 1164, which is longer than the specified 500
Created a chunk of size 688, which is longer than the specified 500
Created a chunk of size 556, which is longer than the specified 500
Created a chunk of size 613, which is longer than the specified 500
Created a chunk of size 734, which is longer than the specified 500
Created a chunk of size 506, which is longer than the specified 500
Created a chunk of size 527, which is longer than the specified 500
Created a chunk of size 536, which is longer than the specified 500
Created a chunk of size 1127, which is longer than the specified 500
Created a chunk of size 847, which is longer t

In [ ]:
texts[190]

Document(page_content="충익공 부조전 전교 \n충익공 부조전을 하사한 전교(傳敎)이다.\n\n6진 개척 \n\n남부 만주지방에는 만주족들이 자리잡고 있었는데, 이들은 두만강과 압록강을 넘어 조선의 국경을 자주 침범하였다. 이 이민족을 '야인'(野人)이라고 불렀는데, 후에는 여진족으로 알려졌다. 고려 때부터 교역을 통하여 회유하기도 하고 무력으로 정벌하기도 하였지만 여진족과의 마찰은 계속 이어졌다. 조선 초에는 영변 이북지방으로 조선의 지배가 미치지 못하고 있었는데, 세종 이후로 조선이 안정되면서 국토가 침탈될 상황에 이른 북방에 주목하게 되었다.\n\n당시 조선의 북방 진지는 정도전이 공주에 설치한 경원부였는데, 계속되는 여진족의 침입으로 방어하기가 어려워졌다. 그는 당시 백성들의 흉년으로 인해 국내문제 해결에 급급하는 조정을 반박하며 세종에게 올린 상소문에 다음과 같은 내용을 남겼다.")

In [ ]:
doc_vector = []
count = 0
for doc in texts:
    count+=1
    doc_vector.append(model.encode(doc.page_content))
    if(count % 50 == 0):
        print("." , end = ' ')

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [ ]:
doc_vector_np = np.array(doc_vector, dtype=np.float32)

In [ ]:
dim = doc_vector_np.shape[-1]

In [ ]:
index2 = faiss.index_factory(dim, 'Flat', faiss.METRIC_INNER_PRODUCT)
print(index2.is_trained)

index2.add(doc_vector_np)
print(f"total number of vectors: {index2.ntotal}")

True
total number of vectors: 4577


In [ ]:
queries_2 = [
    "석기시대란 무엇인가?",
]

query_embeddings_2 = model.encode_queries(queries_2)

dists_2, ids_2 = index2.search(query_embeddings_2.astype(np.float32), k=3)

print(f"query:\t{queries_2[0]}\n\n")
count_2 = 0
for i in range(3):
    print(f"RAG_{count_2}:\t{texts[ids_2[0][i]]}\n")
    count_2 +=1

query:	석기시대란 무엇인가?


RAG_0:	page_content='석기 시대(石器時代, )는 인류 역사를 추적하는 데 고고학 및 역사학에서 쓰이는 낱말로서 인류, 정확히는 인류의 조상인 원인이 돌로 만든 도구를 쓰기 시작한 시대를 일컫는다.

석기는 다양한 종류의 돌로 만든다. 부싯돌과 규질암은 자르는 도구나 무기로 만들어졌고, 현무암이나 사암은 맷돌과 같이 가는 도구로 만들어졌다. 나무, 뼈, 조개껍질, 사슴의 뿔, 그리고 다른 재료들도 필요에 따라 만들어 쓴 것으로 보인다. 이 시기의 말기에는 진흙과 같은 점토도 도기를 만들기 위해 사용되었다. 일련의 기술적 진보가 이후, 동기 시대(Chalcolithic), 청동기 시대, 철기 시대처럼 특징화 된다.'

RAG_1:	page_content='구석기 시대 
석기시대(Paleolithic)는 "old"를 뜻하는 그리스어의 παλαιός, palaios와 "stone"을 뜻하는 λίθος, lithos의 합성어로 "고대 석기 시대"라는 의미로 1865년 고고학자 존 러벅이 만든 용어이다. 이 시기는 석기를 도구로 구분된 선사 시대이다. 지상 위의 인류가 99% 이상 이것을 사용하며, 250만년에서 260만년 전까지 거슬러 올라간다. 호모 하빌리스와 같은 고대 인류가 이러한 석기를 사용하며, 농경이 소개되고, 플라이스토세 말기인 기원전 10000년경에 끝난다. 플라이스토세 기는 중석기 시대나 초기 신석기화 또는 준석기 시대에 끝을 맺는다.'

RAG_2:	page_content='새로운 고고학적 발견으로 새로운 시대와 세부 시대가 추가되어 서로 다른 지역의 상태를 설명하기 위해서 다른 도해가 개발되었다. 석기 시대의 보다 현대적인 시대구분은 구석기 시대에서 신석기 시대를 다음과 같이 분류를 한다.
 플라이스토세 기(홍적세 또는 갱신세) - 두터운 빙하기
 구석기 시대 (Paleolithic)
 홀로세 기(충적세 또는 현세) - 현대적인 날씨
 중석기 시대 (Mesolithic) 또는 준석기 시대(Epip

In [ ]:
faiss.write_index(index2, "index_faiss")

In [ ]:
model = FlagModel('BAAI/bge-m3',
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                  use_fp16=True)

In [ ]:
index3 = faiss.read_index("index_faiss")

In [ ]:
queries_2 = [
    "석기시대란 무엇인가?",
]

query_embeddings_2 = model.encode_queries(queries_2)

dists_3, ids_3 = index3.search(query_embeddings_2.astype(np.float32), k=3)

print(f"query:\t{queries_2[0]}\n\n")
count_2 = 0
for i in range(3):
    print(f"RAG_{count_2}:\t{texts[ids_3[0][i]]}\n")
    count_2 +=1

query:	석기시대란 무엇인가?


RAG_0:	page_content='석기 시대(石器時代, )는 인류 역사를 추적하는 데 고고학 및 역사학에서 쓰이는 낱말로서 인류, 정확히는 인류의 조상인 원인이 돌로 만든 도구를 쓰기 시작한 시대를 일컫는다.

석기는 다양한 종류의 돌로 만든다. 부싯돌과 규질암은 자르는 도구나 무기로 만들어졌고, 현무암이나 사암은 맷돌과 같이 가는 도구로 만들어졌다. 나무, 뼈, 조개껍질, 사슴의 뿔, 그리고 다른 재료들도 필요에 따라 만들어 쓴 것으로 보인다. 이 시기의 말기에는 진흙과 같은 점토도 도기를 만들기 위해 사용되었다. 일련의 기술적 진보가 이후, 동기 시대(Chalcolithic), 청동기 시대, 철기 시대처럼 특징화 된다.'

RAG_1:	page_content='구석기 시대 
석기시대(Paleolithic)는 "old"를 뜻하는 그리스어의 παλαιός, palaios와 "stone"을 뜻하는 λίθος, lithos의 합성어로 "고대 석기 시대"라는 의미로 1865년 고고학자 존 러벅이 만든 용어이다. 이 시기는 석기를 도구로 구분된 선사 시대이다. 지상 위의 인류가 99% 이상 이것을 사용하며, 250만년에서 260만년 전까지 거슬러 올라간다. 호모 하빌리스와 같은 고대 인류가 이러한 석기를 사용하며, 농경이 소개되고, 플라이스토세 말기인 기원전 10000년경에 끝난다. 플라이스토세 기는 중석기 시대나 초기 신석기화 또는 준석기 시대에 끝을 맺는다.'

RAG_2:	page_content='새로운 고고학적 발견으로 새로운 시대와 세부 시대가 추가되어 서로 다른 지역의 상태를 설명하기 위해서 다른 도해가 개발되었다. 석기 시대의 보다 현대적인 시대구분은 구석기 시대에서 신석기 시대를 다음과 같이 분류를 한다.
 플라이스토세 기(홍적세 또는 갱신세) - 두터운 빙하기
 구석기 시대 (Paleolithic)
 홀로세 기(충적세 또는 현세) - 현대적인 날씨
 중석기 시대 (Mesolithic) 또는 준석기 시대(Epip

In [ ]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

sentences_1 = ["What is BGE M3?", "Defination of BM25"]
sentences_2 = ["BGE M3 is an embedding model supporting dense retrieval, lexical matching and multi-vector interaction.",
               "BM25 is a bag-of-words retrieval function that ranks a set of documents based on the query terms appearing in each document"]

In [ ]:
# If you don't need such a long length of 8192 input tokens, you can set max_length to a smaller value to speed up encoding.
embeddings_1 = model.encode(sentences_1, max_length=10)['dense_vecs']
embeddings_2 = model.encode(sentences_2, max_length=100)['dense_vecs']

# compute the similarity scores
s_dense = embeddings_1 @ embeddings_2.T
print(s_dense)

In [ ]:
output_1 = model.encode(sentences_1, return_sparse=True)
output_2 = model.encode(sentences_2, return_sparse=True)

# you can see the weight for each token:
print(model.convert_id_to_token(output_1['lexical_weights']))

In [ ]:
# compute the scores via lexical mathcing
s_lex_10_20 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][0])
s_lex_10_21 = model.compute_lexical_matching_score(output_1['lexical_weights'][0], output_2['lexical_weights'][1])

print(s_lex_10_20)
print(s_lex_10_21)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch, os, faiss
from FlagEmbedding import FlagModel
import numpy as np

In [ ]:
from openai import OpenAI
import math
import time
import json

client = OpenAI(
    base_url="http://localhost:8877/v1",
    api_key="token-abc123",
)

In [ ]:
'''
import datasets

wikidata = datasets.load_dataset('yongchanskii/wiki-korean')
wikidata = wikidata.remove_columns(['id', 'url'])

In [ ]:
#embedding_text = wikidata['train'].select(range(5500,6000))

In [ ]:
'''
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.create_documents(embedding_text['text'])

In [ ]:
index = faiss.read_index("index_faiss")

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [ ]:
model = FlagModel('BAAI/bge-m3',
                  query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                  use_fp16=True)

In [ ]:
question = "2023 대한민국 FA컵 결과 알려줘."

In [ ]:
queries = [question,]

rag_docs = ''
query_embeddings = model.encode_queries(queries)

dists, ids = index.search(query_embeddings.astype(np.float32), k=3)

print(f"query:\t{queries[0]}\n\n")
count = 0
for i in range(3):
    rag_docs += f"후보문장_{count}:\t{texts[ids[0][i]]}\n"
    count +=1

print(rag_docs)

query:	2023 대한민국 FA컵 결과 알려줘.


후보문장_0:	page_content='외부 링크'
후보문장_1:	page_content='역대 선거 결과

각주

외부 링크 
 김동길 교수의 Freedom Watch
 
 조인스 데이터 - 산남 김동길(山南 金東吉) 前 연세대학교 부총장
 두산대백과사전 - 산남 김동길(山南 金東吉) 前 통일국민당 국회의원
 보트 비전 - 산남 김동길(山南 金東吉) 前 자유민주연합 국회의원
 두산대백과사전 - 산남 김동길(山南 金東吉) 前 무소속 국회의원
 김동길TV https://www.youtube.com/channel/UCCCzHfAALDPQ0L11UTsKTWA'
후보문장_2:	page_content='스포츠

축구 
노르트라인베스트팔렌 주는 여러 축구 클럽이 있다: 1. FC 쾰른, 바이어 04 레버쿠젠, 보루시아 도르트문트, 보루시아 묀헨글라트바흐, 아르미니아 빌레펠트, 알레마니아 아헨, 부퍼탈 SV 보루시아, 포르투나 뒤셀도르프, FC 샬케 04, MSV 뒤스부르크, SC 파더보른 07, VfL 보훔

또한 이 주에서 1974년 FIFA 월드컵과 2006년 FIFA 월드컵의 경기가 열렸으며, 2011년 FIFA 여자 월드컵의 경기도 개최했다. 1974년 월드컵에서는 뒤셀도르프에 있는 라인슈타디온, 겔젠키르헨의 파르크슈타디온에서 경기가 열렸으며, 2006년 월드컵에서는 쾰른의 라인에네르기슈타디온, 도르트문트의 지그날 이두나 파르크, 겔젠키르헨의 펠틴스 아레나에서 열렸다. 레버쿠젠의 바이아레나와 보훔에 있는 루르슈타디온이 2011년 여자 FIFA 월드컵의 개최 경기장이 되었다.'



In [ ]:
system_prompt = """You are a helpful assistant. And Answers must be in Korean.
그리고 답변할땐 꼭 다음의 지시 사항을 준수해줘.
1) 질문에 대한 답변 후보 문장들을 자세히 읽고 유저가 물어본 질문에 제시된 정보만 활용해서 질문과 정확성, 관련성, 신뢰성을 종합적으로 고려하여 답변을 만들어 제공해주세요.
2) 그리고 후보문장 중 답변에 활용하는데 가장 적합한 후보가 어떤것 이였는지를 알려 주세요.
3) 후보 문장들 중 질문에 대한 답이 없을 경우 "답변을 찾을 수 없습니다. 좀 더 구체적으로 질문해 주세요." 라고 답하세요."""

user_prompt = question + "\n후보문장: " + rag_docs

In [ ]:
user_prompt

'석기시대란 무엇인가?\n후보문장: 후보문장_0:\tpage_content=\'석기 시대(石器時代, )는 인류 역사를 추적하는 데 고고학 및 역사학에서 쓰이는 낱말로서 인류, 정확히는 인류의 조상인 원인이 돌로 만든 도구를 쓰기 시작한 시대를 일컫는다.\n\n석기는 다양한 종류의 돌로 만든다. 부싯돌과 규질암은 자르는 도구나 무기로 만들어졌고, 현무암이나 사암은 맷돌과 같이 가는 도구로 만들어졌다. 나무, 뼈, 조개껍질, 사슴의 뿔, 그리고 다른 재료들도 필요에 따라 만들어 쓴 것으로 보인다. 이 시기의 말기에는 진흙과 같은 점토도 도기를 만들기 위해 사용되었다. 일련의 기술적 진보가 이후, 동기 시대(Chalcolithic), 청동기 시대, 철기 시대처럼 특징화 된다.\'\n후보문장_1:\tpage_content=\'구석기 시대 \n석기시대(Paleolithic)는 "old"를 뜻하는 그리스어의 παλαιός, palaios와 "stone"을 뜻하는 λίθος, lithos의 합성어로 "고대 석기 시대"라는 의미로 1865년 고고학자 존 러벅이 만든 용어이다. 이 시기는 석기를 도구로 구분된 선사 시대이다. 지상 위의 인류가 99% 이상 이것을 사용하며, 250만년에서 260만년 전까지 거슬러 올라간다. 호모 하빌리스와 같은 고대 인류가 이러한 석기를 사용하며, 농경이 소개되고, 플라이스토세 말기인 기원전 10000년경에 끝난다. 플라이스토세 기는 중석기 시대나 초기 신석기화 또는 준석기 시대에 끝을 맺는다.\'\n후보문장_2:\tpage_content=\'새로운 고고학적 발견으로 새로운 시대와 세부 시대가 추가되어 서로 다른 지역의 상태를 설명하기 위해서 다른 도해가 개발되었다. 석기 시대의 보다 현대적인 시대구분은 구석기 시대에서 신석기 시대를 다음과 같이 분류를 한다.\n 플라이스토세 기(홍적세 또는 갱신세) - 두터운 빙하기\n 구석기 시대 (Paleolithic)\n 홀로세 기(충적세 또는 현세) - 현대적인 날씨\n 중석기 시대 (Mesolithi

In [ ]:
messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
assistant_response = generate_response(messages)

2023 대한민국 FA컵 결과를 알려드릴 수 없습니다. 좀 더 구체적으로 질문해 주세요.

적합한 후보문장이 없습니다.

inference time: 1.19914 sec 




In [ ]:
def generate_response(message_list):
    completion = client.chat.completions.create(
        model = "ISTA-DASLab/gemma-3-27b-it-GPTQ-4b-128g",
        messages = message_list,
        max_tokens=1024,
        frequency_penalty=0.3,
        temperature=0.6,
        stream=True,
    )

    final_answer = []
    assistant_response = ""

    start = time.time()

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content

        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content

    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")
    return assistant_response